In [1]:
import os
import sys
import pyowm
import pandas as pd
import numpy as np
import json
import requests
from pyowm.owm import OWM
from pyowm.weatherapi25.forecast import Forecast

In [2]:
root_dir = os.path.join(os.getcwd(), '..')
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(root_dir)
sys.path.append(src_dir)

from conf.auth import OWM_api_key

In [3]:
owm = pyowm.OWM(OWM_api_key)

In [4]:
park_loc = pd.DataFrame({
    'name': ['Prospect Park', 'Wooster Square'],
    'lat': [40.662551, 41.304745],
    'long': [-73.969256, -72.917757]})

In [5]:
park_name = 'Prospect Park'
park_lat = park_loc.loc[park_loc.name == park_name, 'lat'].iloc[0]
park_long = park_loc.loc[park_loc.name == park_name, 'long'].iloc[0]

mgr = owm.weather_manager()
park_forecast = mgr.forecast_at_coords(lat=float(park_lat), lon=float(park_long), interval='3h').forecast
park_forecast

<pyowm.weatherapi25.forecast.Forecast - reception_time=2020-04-22 16:46:46+00, interval=3h>

In [6]:
 park_forecast.reception_time()

1587574006

In [7]:
park_forecast.get(1).to_dict()

{'reference_time': 1587589200,
 'sunset_time': 0,
 'sunrise_time': 0,
 'clouds': 17,
 'rain': {},
 'snow': {},
 'wind': {'speed': 9.25, 'deg': 279},
 'humidity': 39,
 'pressure': {'press': 1012, 'sea_level': 1012},
 'temperature': {'temp': 283.3,
  'temp_kf': 0.53,
  'temp_max': 283.3,
  'temp_min': 282.77,
  'feels_like': 274.42},
 'status': 'Clouds',
 'detailed_status': 'few clouds',
 'weather_code': 801,
 'weather_icon_name': '02d',
 'visibility_distance': None,
 'dewpoint': None,
 'humidex': None,
 'heat_index': None,
 'utc_offset': None}

In [8]:
OWM_forecast_cols = [
    'park_name', 'reception_time', 'reference_time', 'sunset_time',
    'sunrise_time', 'clouds', 'rain', 'snow', 'wind_speed',
    'wind_deg', 'humidity', 'press', 'press_sea', 'temp',
    'temp_feels', 'temp_max', 'temp_min', 'status', 'detailed_status']

In [9]:
def forecast_to_df(park_forecast, t_sunrise, t_sunset, tz_correct):
    ''' Takes a forecast object and sunrise/sunset time,
        Returns datafram of relevant info
    '''
    # Neet to adjust sunrise/set times, should only go
    # into columns of the same day (or just use last day in window?)
    
    forecast_df = pd.DataFrame(columns=OWM_forecast_cols)

    for x in park_forecast:
        if (t_sunrise == 0) or (t_sunset == 0):
            x_sunrise = np.nan
            x_sunset = np.nan
        else:
            x_sunrise = t_sunrise + tz_correct
            x_sunset = t_sunset + tz_correct

        x_rain = 0
        if x.rain:
            x_rain = x.rain['3h']

        x_snow = 0
        if x.snow:
            x_snow = x.snow['3h']

        to_add_dict = {
            'park_name': [park_name],
            'reception_time': [park_forecast.reception_time('unix') + tz_correct],
            'reference_time': [x.reference_time() + tz_correct],
            'sunrise_time': [x_sunrise],
            'sunset_time': [x_sunset],
            'clouds': [x.clouds],
            'rain': [x_rain],
            'snow': [x_snow],
            'wind_speed': [x.wind(unit='meters_sec')['speed']],
            'wind_deg': [x.wind(unit='meters_sec')['deg']],
            'humidity': [x.humidity],
            'press': [x.pressure['press']],
            'press_sea': [x.pressure['sea_level']],
            'temp': [x.temperature(unit='fahrenheit')['temp']],
            'temp_feels': [x.temperature(unit='fahrenheit')['feels_like']],
            'temp_max': [x.temperature(unit='fahrenheit')['temp_max']],
            'temp_min': [x.temperature(unit='fahrenheit')['temp_min']],
            'status': [x.status],
            'detailed_status': [x.detailed_status]}
        
        to_add = pd.DataFrame(to_add_dict)
        forecast_df = forecast_df.append(to_add, ignore_index=True)
    
    return forecast_df

In [10]:
forcast_df = forecast_to_df(park_forecast, 0, 0, 10)
forcast_df.head()

,park_name,reception_time,reference_time,sunset_time,sunrise_time,clouds,rain,snow,wind_speed,wind_deg,humidity,press,press_sea,temp,temp_feels,temp_max,temp_min,status,detailed_status
0,Prospect Park,1587574016,1587578410,NaN,NaN,51,0,0,8.51,284,34,1011,1011,48.52,32.94,48.52,46.26,Clouds,broken clouds
1,Prospect Park,1587574016,1587589210,NaN,NaN,17,0,0,9.25,279,39,1012,1012,50.27,34.29,50.27,49.32,Clouds,few clouds
2,Prospect Park,1587574016,1587600010,NaN,NaN,6,0,0,7.84,283,46,1014,1014,48.97,35.11,48.97,48.69,Clear,clear sky
3,Prospect Park,1587574016,1587610810,NaN,NaN,1,0,0,6.93,315,52,1017,1017,46.02,33.35,46.02,45.99,Clear,clear sky
4,Prospect Park,1587574016,1587621610,NaN,NaN,1,0,0,5.50,312,58,1017,1017,42.55,31.60,42.55,42.55,Clear,clear sky


Errors in pyowm means I can't get sunrise/sunset time directly, so moving to:

- Calling the api manually for the forcast JSON
- Extracting the sunrise/sunset time
- Passing the JSON back into the pyowm parser then use the work from above

In [11]:
api_url = 'http://api.openweathermap.org/data/2.5/forecast'
payload = {
    'lat': park_lat,
    'lon': park_long,
    'appid': OWM_api_key}
r = requests.get(api_url, params=payload)

In [12]:
d = json.loads(r.text)

In [13]:
sunrise_time = d['city']['sunrise']
sunset_time = d['city']['sunset']
tz_correct = d['city']['timezone']

In [14]:
# weather_df = forecast_to_df(forecastparser.ForecastParser().parse_JSON(r.text), sunrise_time, sunset_time, tz_correct)
weather_df = forecast_to_df(Forecast.from_dict(json.loads(r.text)), sunrise_time, sunset_time, tz_correct)

In [15]:
weather_df.head(5)

,park_name,reception_time,reference_time,sunset_time,sunrise_time,clouds,rain,snow,wind_speed,wind_deg,humidity,press,press_sea,temp,temp_feels,temp_max,temp_min,status,detailed_status
0,Prospect Park,1587559606,1587564000,1587584531,1587535550,51,0,0,8.51,284,34,1011,1011,48.70,33.13,48.70,46.26,Clouds,broken clouds
1,Prospect Park,1587559606,1587574800,1587584531,1587535550,17,0,0,9.25,279,39,1012,1012,50.34,34.36,50.34,49.32,Clouds,few clouds
2,Prospect Park,1587559606,1587585600,1587584531,1587535550,6,0,0,7.84,283,46,1014,1014,48.99,35.13,48.99,48.69,Clear,clear sky
3,Prospect Park,1587559606,1587596400,1587584531,1587535550,1,0,0,6.93,315,52,1017,1017,46.02,33.35,46.02,45.99,Clear,clear sky
4,Prospect Park,1587559606,1587607200,1587584531,1587535550,1,0,0,5.50,312,58,1017,1017,42.55,31.60,42.55,42.55,Clear,clear sky


The dataframe weather_df is what I'll be adding to the log of forcast data.

In [16]:
weather_df.to_csv('example_weather_df.csv')